# 4. Create Azure Virtual Machine as IoT Edge device

In [1]:
# In case the sample requires a GPU, use a GPU tier VM size such as:
#           Standard_NC6
#
# In case you will not use additional accelerator but just CPU, you may prefer raw CPU VM such as:
#           Standard_DS3_v2
#
# Set below variable according to your sample's requirements.

vm_size="Standard_NC6"
vm_size="Standard_DS3_v2"

## 4.1. Get global variables
We will read the previously stored variables

In [ ]:
from dotenv import set_key, get_key, find_dotenv
envPath = find_dotenv(raise_error_if_not_found=True)

resourceLocation = get_key(envPath, "resourceLocation")
resourceGroupName = get_key(envPath, "resourceGroupName")
iotEdgeDeviceConnString = get_key(envPath, "iotEdgeDeviceConnString")
# We will use IoT Edge Device ID also as VM name
iotDeviceId = get_key(envPath, "iotDeviceId")

# We get current username so will use it to create SSH keys for connecting the VM
import os
userName = os.environ['USER']

## 4.2. Create a Virtual Machine
With following steps, we will execute a shell script to create an Ubuntu VM in your Azure subscription.  

### 4.2.1 Set parameters to be used for creating the Virtual Machine
https://docs.microsoft.com/en-us/azure/virtual-machines/windows/quick-create-cli

In [ ]:
# Select VM size per your speed requirements. https://docs.microsoft.com/en-us/azure/virtual-machines/sizes-general 
dns_name = iotDeviceId
public_ip_name = iotDeviceId + 'publicip'
vnet_name=iotDeviceId + 'vnet'
subnet_name=iotDeviceId + 'subnet'
vnet_prefix="192.168.0.0/16"
subnet_name="FrontEnd"
subnet_prefix="192.168.1.0/24"
nsg_name=iotDeviceId + 'nsg'
nic_name=iotDeviceId + 'nic'

# Static DNS name of the VM.
vm_dns_name= iotDeviceId + "." + resourceLocation + ".cloudapp.azure.com"

In [ ]:
%%bash --out output -s "$iotDeviceId" "$userName" "$vm_dns_name"

# -----------------------
# Create SSH Keys - !!! DONT RUN THIS CELL IN CASE YOU ALREADY CREATED THE VM. IT WILL RESET THE PUBLIC KEYS!!!
# -----------------------
# Create public/private ssh keys.
yes y | ssh-keygen -b 2048 -t rsa -f ~/.ssh/$1"_id_rsa" -C $2@$3 -q -N '' >/dev/null

# Remove prev. trusted host with same name (IP may be changed)
if [ -e ~/.ssh/known_hosts ]; then
ssh-keygen -f ~/.ssh/known_hosts -R $3
fi

In [ ]:
%%bash -s "$resourceGroupName" "$resourceLocation" "$public_ip_name" "$dns_name" "$vnet_name" "$vnet_prefix" "$subnet_name" "$subnet_prefix" "$nsg_name" "$nic_name" "$iotDeviceId" "$vm_size"

# Create resource group
az group create                                                                             \
    --name "$1"                                                                             \
    --location "$2"

# Create a public IP address resource with a static IP address
az network public-ip create                                                                 \
   --name "$3"                                                                              \
   --resource-group "$1"                                                                    \
   --location "$2"                                                                          \
   --allocation-method Static                                                               \
   --dns-name "$4"                                                                                   

# Create a virtual network with one subnet
az network vnet create                                                                      \
   --name "$5"                                                                              \
   --resource-group "$1"                                                                    \
   --location "$2"                                                                          \
   --address-prefix "$6"                                                                    \
   --subnet-name "$7"                                                                       \
   --subnet-prefix "$8"                                                                         


az network nsg create                                                                       \
   --name "$9"                                                                              \
   --resource-group "$1"                                                                  

# Open SSH port
az network nsg rule create                                                                  \
   --resource-group "$1"                                                                    \
   --nsg-name "$9"                                                                          \
   --name "Default SSH"                                                                     \
   --destination-port-ranges 22                                                             \
   --protocol Tcp                                                                           \
   --access Allow                                                                           \
   --priority 1020

# Create a network interface connected to the VNet with a static private IP address 
# and associate the public IP address resource to the NIC.
az network nic create                                                                       \
   --name "${10}"                                                                           \
   --resource-group "$1"                                                                    \
   --location "$2"                                                                          \
   --subnet "$7"                                                                            \
   --vnet-name "$5"                                                                         \
   --public-ip-address "$3"                                                                 \
   --network-security-group "$9"                                                                     

# Create the VM
# For image param: Ubuntu 16.04-LTS image -> https://docs.microsoft.com/en-us/azure/virtual-machines/linux/cli-ps-findimage
# Instead of Password login, we are uploading Private SSH key. Update the path accrodingly if needed
az vm create                                                                                \
   --name "${11}"                                                                           \
   --resource-group "$1"                                                                    \
   --location "$2"                                                                          \
   --storage-sku Standard_LRS                                                               \
   --os-disk-name "${11}""_osdisk"                                                          \
   --image "Canonical:UbuntuServer:18.04-LTS:latest"                                        \
   --size "${12}"                                                                           \
   --nics "${10}"                                                                           \
   --authentication-type ssh                                                                \
   --ssh-key-value "$(< ~/.ssh/"${11}"_id_rsa.pub)"

In [ ]:
%%bash -s "$iotDeviceId" "$userName" "$vm_dns_name"

# Install ssh keys to remote machine
ssh -i ~/.ssh/$1"_id_rsa" -o "StrictHostKeyChecking no" $2@$3 "echo 'SSH key transferred.'"

### Restart the VM

In [ ]:
%%bash  -s "$iotDeviceId" "$userName" "$vm_dns_name"
ssh -i ~/.ssh/$1"_id_rsa" $2@$3 << EOF
sudo -b bash -c 'sleep 5; reboot' &>/dev/null;
EOF

### Print out the SSH connection string for this VM

In [ ]:
%%bash --out sshstring

# If needed, use the output command to SSH into the VM
echo ssh -i ~/.ssh/$iotDeviceId"_id_rsa" $userName@$vm_dns_name

In [ ]:
# save the ssh conn string into .env file
from dotenv import set_key, get_key, find_dotenv
envPath = find_dotenv(raise_error_if_not_found=True)

set_key(envPath, "sshstring", sshstringsshstring)